<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/NFC_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This notebook is a tutorial for the article: 

[Neural Collaborative Filtering](https://arxiv.org/pdf/1708.05031.pdf)

# Explanation

### Matrix Factorization
> ##### Let’s begin with Matrix Factorization. It decomposes the utility matrix into two sub matrices. During prediction, we multiply the two sub-matrices to reconstruct the predicted utility matrix. The utility matrix is factorized such that the loss between the multiplication of these two and the true utility matrix is minimized. One commonly used loss function is mean-squared error.

<img src="https://drive.google.com/uc?id=1ru0sw_iSaDErRpAh549RxupedZDWQY5s" alt="drawing" width="700"/>



<img src="https://drive.google.com/uc?id=1SrMZxzBCjhHH3RsALg9HzwJiF7emAqJW" alt="drawing" width="700"/>


> #### Essentially, each user and item is projected onto a latent space, represented by a latent vector. The more similar the two latent vectors are, the more related the corresponding users’ preference. Since we factorize the utility matrix into the same latent space, we can measure the similarity of any two latent vectors with cosine-similarity or dot product. In fact, the prediction for each user/ item entry is computed by the dot product of the corresponding latent vectors.

<img src="https://drive.google.com/uc?id=1ppD_wElrYifbxf2336mrtu_eGtIspgQg" alt="drawing" width="700"/>

> #### However, the paper argued that dot product limits the expressiveness of user and item latent vectors. Let’s consider the following case. We first focus on the top three rows of this utility matrix.

<img src="https://drive.google.com/uc?id=1Mm2Ma6X45JtM9TVP5AgsZorWhjZP0JPt" alt="drawing" width="700"/>


## Neural Collaborative Filtering
> #### The paper proposed Neural Collaborative Filtering as shown in the graph below. In the input layer, the user and item are one-hot encoded. Then, they are mapped to the hidden space with embedding layers accordingly. The Neural FC layer can be any kind neuron connections. Multiple layer perceptron, for example, can be placed here. It claims that with the complicated connection and non-linearity in the Neural CF layers, this model is capable of properly estimating the complex interactions between user and item in the latent space.

<img src="https://drive.google.com/uc?id=1zplOPLxNYY4aj_j4qcj6_xQoNYkLjMlS" alt="drawing" width="700"/>


> #### The NCF is a generalization of Matrix Factorization. We first replace Neural CF layer with a multiplication layer, which performs element-wise multiplication on the two inputs. Then, we set the weight from the multiplication layer to the output layer to be a fixed unit matrix (matrix of all ones) of dimension $K \times 1$ with linear activation function.


<img src="https://drive.google.com/uc?id=1cLUSxl342LRyldIP7WcsratPE1HY7Y62" alt="drawing" width="700"/>



> #### Then, we have the following equations. The prediction of the unobserved interaction $\hat{y}_{ui}$ denotes the predicted value of the $(u, i)$ entry on the reconstructed utility matrix. $L$ is the linear activation function, while $⊙$ denotes the element-wise multiplication operation. $p_u$ and $q_i$ are the latent vector of user and item, respectively, and $J$ is the unit matrix of dimension $(K,1)$. Since $J$ is a unit matrix, inside the linear function becomes the inner product between latent vector $p_u$ and $q_i$. Furthermore, because the input and output are the same for linear function, it boils down to the last line. The predicted label is the inner product of the corresponding user and item latent vector. This equation is identical to that shown in the Matrix Factorization section. Thus, this proves that Matrix Factorization is a special case of NCF.


<img src="https://drive.google.com/uc?id=1bDx93lQXu7DCfOGAXg-6u6-EVy005CZY" alt="drawing" width="700"/>


## NeuMF

> #### In order to introduce additional non-linearity, the final model proposed, NeuMF, includes a Mutliple-layer Perceptron (MLP) module apart from the Generalized Matrix Factorization (GMP) layer. The output of GMF and MLP modules are concatenated and connected with the sigmoid activation output layer.

<img src="https://drive.google.com/uc?id=10GVO_vb3wBnlMz5kQi8bVIJiMJ01z4-j" alt="drawing" width="700"/>


## Model

In [ ]:
def __init__(self, config):
    super(NeuMF, self).__init__()

    #mf part
    self.embedding_user_mf = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim_mf)
    self.embedding_item_mf = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim_mf)

    #mlp part
    self.embedding_user_mlp = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim_mlp)
    self.embedding_item_mlp = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim_mlp)

    self.fc_layers = torch.nn.ModuleList()
    for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
        self.fc_layers.append(torch.nn.Linear(in_size, out_size))

    self.logits = torch.nn.Linear(in_features=config['layers'][-1] + config['latent_dim_mf']  , out_features=1)
    self.sigmoid = torch.nn.Sigmoid()

> #### The paper claimed that using separate embedding layer for MLP and GMF yield better performance. Therefore, we define the designated embedding layer for these two modules. In addition, ModuleList is leveraged for building multiple layers of perceptron.

In [ ]:
def forward(self, user_indices, item_indices, titles):
    user_embedding_mlp = self.embedding_user_mlp(user_indices)
    item_embedding_mlp = self.embedding_item_mlp(item_indices)
    user_embedding_mf = self.embedding_user_mf(user_indices)
    item_embedding_mf = self.embedding_item_mf(item_indices)

    #### mf part
    mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)
    mf_vector = torch.nn.Dropout(self.config.dropout_rate_mf)(mf_vector)

    #### mlp part        
    mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector

    for idx, _ in enumerate(range(len(self.fc_layers))):
        mlp_vector = self.fc_layers[idx](mlp_vector)
        mlp_vector = torch.nn.ReLU()(mlp_vector)
    mlp_vector = torch.nn.Dropout(self.config.dropout_rate_mlp)(mlp_vector)

    vector = torch.cat([mlp_vector, mf_vector], dim=-1)
    logits = self.logits(vector)
    output = self.sigmoid(logits)
    return output

## Results


<img src="https://drive.google.com/uc?id=1XZtLeBYiNhQ5MUL4RimK5_3lhTzSczuu" alt="drawing" width="1000"/>




<img src="https://drive.google.com/uc?id=1FZOZAMB3uYURFF6jTjYyOAzP1xIRt-az" alt="drawing" width="700"/>



# Code and Implementation with MovieLens 1M

In [ ]:
import math
import torch
import random
import numpy as np
import pandas as pd
from copy import deepcopy
from torch.autograd import Variable
# from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader, Dataset

random.seed(0)

In [ ]:
# Checkpoints
def save_checkpoint(model, model_dir):
    torch.save(model.state_dict(), model_dir)

def resume_checkpoint(model, model_dir, device_id):
    state_dict = torch.load(model_dir,
                            map_location=lambda storage, loc: storage.cuda(device=device_id))  # ensure all storage are on gpu
    model.load_state_dict(state_dict)

# Hyper params
def use_cuda(enabled, device_id=0):
    if enabled:
        assert torch.cuda.is_available(), 'CUDA is not available'
        torch.cuda.set_device(device_id)

def use_optimizer(network, params):
    if params['optimizer'] == 'sgd':
        optimizer = torch.optim.SGD(network.parameters(),
                                    lr=params['sgd_lr'],
                                    momentum=params['sgd_momentum'],
                                    weight_decay=params['l2_regularization'])
    elif params['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(network.parameters(), 
                                                          lr=params['adam_lr'],
                                                          weight_decay=params['l2_regularization'])
    elif params['optimizer'] == 'rmsprop':
        optimizer = torch.optim.RMSprop(network.parameters(),
                                        lr=params['rmsprop_lr'],
                                        alpha=params['rmsprop_alpha'],
                                        momentum=params['rmsprop_momentum'])
    return optimizer

In [ ]:
class UserItemRatingDataset(Dataset):
    """Wrapper, convert <user, item, rating> Tensor into Pytorch Dataset"""
    def __init__(self, user_tensor, item_tensor, target_tensor):
        """
        args:

            target_tensor: torch.Tensor, the corresponding rating for <user, item> pair
        """
        self.user_tensor = user_tensor
        self.item_tensor = item_tensor
        self.target_tensor = target_tensor

    def __getitem__(self, index):
        return self.user_tensor[index], self.item_tensor[index], self.target_tensor[index]

    def __len__(self):
        return self.user_tensor.size(0)


class SampleGenerator(object):
    """Construct dataset for NCF"""

    def __init__(self, ratings):
        """
        args:
            ratings: pd.DataFrame, which contains 4 columns = ['userId', 'itemId', 'rating', 'timestamp']
        """
        assert 'userId' in ratings.columns
        assert 'itemId' in ratings.columns
        assert 'rating' in ratings.columns

        self.ratings = ratings
        # explicit feedback using _normalize and implicit using _binarize
        # self.preprocess_ratings = self._normalize(ratings)
        self.preprocess_ratings = self._binarize(ratings)
        self.user_pool = set(self.ratings['userId'].unique())
        self.item_pool = set(self.ratings['itemId'].unique())
        # create negative item samples for NCF learning
        self.negatives = self._sample_negative(ratings)
        self.train_ratings, self.test_ratings = self._split_loo(self.preprocess_ratings)

    def _normalize(self, ratings):
        """normalize into [0, 1] from [0, max_rating], explicit feedback"""
        ratings = deepcopy(ratings)
        max_rating = ratings.rating.max()
        ratings['rating'] = ratings.rating * 1.0 / max_rating
        return ratings
    
    def _binarize(self, ratings):
        """binarize into 0 or 1, imlicit feedback"""
        ratings = deepcopy(ratings)
        ratings['rating'][ratings['rating'] > 0] = 1.0
        return ratings

    def _split_loo(self, ratings):
        """leave one out train/test split """
        ratings['rank_latest'] = ratings.groupby(['userId'])['timestamp'].rank(method='first', ascending=False)
        test = ratings[ratings['rank_latest'] == 1]
        train = ratings[ratings['rank_latest'] > 1]
        assert train['userId'].nunique() == test['userId'].nunique()
        return train[['userId', 'itemId', 'rating']], test[['userId', 'itemId', 'rating']]

    def _sample_negative(self, ratings):
        """return all negative items & 100 sampled negative items"""
        interact_status = ratings.groupby('userId')['itemId'].apply(set).reset_index().rename(
            columns={'itemId': 'interacted_items'})
        interact_status['negative_items'] = interact_status['interacted_items'].apply(lambda x: self.item_pool - x)
        interact_status['negative_samples'] = interact_status['negative_items'].apply(lambda x: random.sample(x, 99))
        return interact_status[['userId', 'negative_items', 'negative_samples']]

    def instance_a_train_loader(self, num_negatives, batch_size):
        """instance train loader for one training epoch"""
        users, items, ratings = [], [], []
        train_ratings = pd.merge(self.train_ratings, self.negatives[['userId', 'negative_items']], on='userId')
        train_ratings['negatives'] = train_ratings['negative_items'].apply(lambda x: random.sample(x, num_negatives))
        for row in train_ratings.itertuples():
            users.append(int(row.userId))
            items.append(int(row.itemId))
            ratings.append(float(row.rating))
            for i in range(num_negatives):
                users.append(int(row.userId))
                items.append(int(row.negatives[i]))
                ratings.append(float(0))  # negative samples get 0 rating
        dataset = UserItemRatingDataset(user_tensor=torch.LongTensor(users),
                                        item_tensor=torch.LongTensor(items),
                                        target_tensor=torch.FloatTensor(ratings))
        return DataLoader(dataset, batch_size=batch_size, shuffle=True)

    @property
    def evaluate_data(self):
        """create evaluate data"""
        test_ratings = pd.merge(self.test_ratings, self.negatives[['userId', 'negative_samples']], on='userId')
        test_users, test_items, negative_users, negative_items = [], [], [], []
        for row in test_ratings.itertuples():
            test_users.append(int(row.userId))
            test_items.append(int(row.itemId))
            for i in range(len(row.negative_samples)):
                negative_users.append(int(row.userId))
                negative_items.append(int(row.negative_samples[i]))
        return [torch.LongTensor(test_users), torch.LongTensor(test_items), torch.LongTensor(negative_users),
                torch.LongTensor(negative_items)]

In [ ]:
class MetronAtK(object):
    def __init__(self, top_k):
        self._top_k = top_k
        self._subjects = None  # Subjects which we ran evaluation on

    @property
    def top_k(self):
        return self._top_k

    @top_k.setter
    def top_k(self, top_k):
        self._top_k = top_k

    @property
    def subjects(self):
        return self._subjects

    @subjects.setter
    def subjects(self, subjects):
        """
        args:
            subjects: list, [test_users, test_items, test_scores, negative users, negative items, negative scores]
        """
        assert isinstance(subjects, list)
        test_users, test_items, test_scores = subjects[0], subjects[1], subjects[2]
        neg_users, neg_items, neg_scores = subjects[3], subjects[4], subjects[5]
        # the golden set
        test = pd.DataFrame({'user': test_users,
                             'test_item': test_items,
                             'test_score': test_scores})
        # the full set
        full = pd.DataFrame({'user': neg_users + test_users,
                            'item': neg_items + test_items,
                            'score': neg_scores + test_scores})
        full = pd.merge(full, test, on=['user'], how='left')
        # rank the items according to the scores for each user
        full['rank'] = full.groupby('user')['score'].rank(method='first', ascending=False)
        full.sort_values(['user', 'rank'], inplace=True)
        self._subjects = full

    def cal_hit_ratio(self):
        """Hit Ratio @ top_K"""
        full, top_k = self._subjects, self._top_k
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]  # golden items hit in the top_K items
        return len(test_in_top_k) * 1.0 / full['user'].nunique()

    def cal_ndcg(self):
        full, top_k = self._subjects, self._top_k
        top_k = full[full['rank']<=top_k]
        test_in_top_k =top_k[top_k['test_item'] == top_k['item']]
        test_in_top_k['ndcg'] = test_in_top_k['rank'].apply(lambda x: math.log(2) / math.log(1 + x)) # the rank starts from 1
        return test_in_top_k['ndcg'].sum() * 1.0 / full['user'].nunique()

In [ ]:
class Engine(object):
    """Meta Engine for training & evaluating NCF model

    Note: Subclass should implement self.model !
    """

    def __init__(self, config):
        self.config = config  # model configuration
        self._metron = MetronAtK(top_k=10)
        self.opt = use_optimizer(self.model, config)
        # explicit feedback
        # self.crit = torch.nn.MSELoss()
        # implicit feedback
        self.crit = torch.nn.BCELoss()

    def train_single_batch(self, users, items, ratings):
        assert hasattr(self, 'model'), 'Please specify the exact model !'
        if self.config['use_cuda'] is True:
            users, items, ratings = users.cuda(), items.cuda(), ratings.cuda()
        self.opt.zero_grad()
        ratings_pred = self.model(users, items)
        loss = self.crit(ratings_pred.view(-1), ratings)
        loss.backward()
        self.opt.step()
        loss = loss.item()
        return loss

    def train_an_epoch(self, train_loader, epoch_id):
        assert hasattr(self, 'model'), 'Please specify the exact model !'
        self.model.train()
        total_loss = 0
        for batch_id, batch in enumerate(train_loader):
            assert isinstance(batch[0], torch.LongTensor)
            user, item, rating = batch[0], batch[1], batch[2]
            rating = rating.float()
            loss = self.train_single_batch(user, item, rating)
            if batch_id % 400 == 0:
                print('[Training Epoch {}] Batch {}, Loss {}'.format(epoch_id, batch_id, loss))
            total_loss += loss
        # self._writer.add_scalar('model/loss', total_loss, epoch_id)

    def evaluate(self, evaluate_data, epoch_id):
        assert hasattr(self, 'model'), 'Please specify the exact model !'
        self.model.eval()
        with torch.no_grad():
            test_users, test_items = evaluate_data[0], evaluate_data[1]
            negative_users, negative_items = evaluate_data[2], evaluate_data[3]
            if self.config['use_cuda'] is True:
                test_users = test_users.cuda()
                test_items = test_items.cuda()
                negative_users = negative_users.cuda()
                negative_items = negative_items.cuda()
            test_scores = self.model(test_users, test_items)
            negative_scores = self.model(negative_users, negative_items)
            if self.config['use_cuda'] is True:
                test_users = test_users.cpu()
                test_items = test_items.cpu()
                test_scores = test_scores.cpu()
                negative_users = negative_users.cpu()
                negative_items = negative_items.cpu()
                negative_scores = negative_scores.cpu()
            self._metron.subjects = [test_users.data.view(-1).tolist(),
                                 test_items.data.view(-1).tolist(),
                                 test_scores.data.view(-1).tolist(),
                                 negative_users.data.view(-1).tolist(),
                                 negative_items.data.view(-1).tolist(),
                                 negative_scores.data.view(-1).tolist()]
        hit_ratio, ndcg = self._metron.cal_hit_ratio(), self._metron.cal_ndcg()
        # self._writer.add_scalar('performance/HR', hit_ratio, epoch_id)
        # self._writer.add_scalar('performance/NDCG', ndcg, epoch_id)
        print('[Evluating Epoch {}] HR = {:.4f}, NDCG = {:.4f}'.format(epoch_id, hit_ratio, ndcg))
        return hit_ratio, ndcg

    def save(self, alias, epoch_id, hit_ratio, ndcg):
        assert hasattr(self, 'model'), 'Please specify the exact model !'
        model_dir = self.config['model_dir'].format(alias, epoch_id, hit_ratio, ndcg)
        save_checkpoint(self.model, model_dir)

In [ ]:
class GMF(torch.nn.Module):
    def __init__(self, config):
        super(GMF, self).__init__()
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim']

        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.affine_output = torch.nn.Linear(in_features=self.latent_dim, out_features=1)
        self.logistic = torch.nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        element_product = torch.mul(user_embedding, item_embedding)
        logits = self.affine_output(element_product)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass


class GMFEngine(Engine):
    """Engine for training & evaluating GMF model"""
    def __init__(self, config):
        self.model = GMF(config)
        if config['use_cuda'] is True:
            use_cuda(True, config['device_id'])
            self.model.cuda()
        super(GMFEngine, self).__init__(config)

In [ ]:
class MLP(torch.nn.Module):
    def __init__(self, config):
        super(MLP, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim = config['latent_dim']

        self.embedding_user = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim)
        self.embedding_item = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim)

        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = torch.nn.Linear(in_features=config['layers'][-1], out_features=1)
        self.logistic = torch.nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding = self.embedding_user(user_indices)
        item_embedding = self.embedding_item(item_indices)
        vector = torch.cat([user_embedding, item_embedding], dim=-1)  # the concat latent vector
        for idx, _ in enumerate(range(len(self.fc_layers))):
            vector = self.fc_layers[idx](vector)
            vector = torch.nn.ReLU()(vector)
            # vector = torch.nn.BatchNorm1d()(vector)
            # vector = torch.nn.Dropout(p=0.5)(vector)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

    def load_pretrain_weights(self):
        """Loading weights from trained GMF model"""
        config = self.config
        gmf_model = GMF(config)
        if config['use_cuda'] is True:
            gmf_model.cuda()
        resume_checkpoint(gmf_model, model_dir=config['pretrain_mf'], device_id=config['device_id'])
        self.embedding_user.weight.data = gmf_model.embedding_user.weight.data
        self.embedding_item.weight.data = gmf_model.embedding_item.weight.data


class MLPEngine(Engine):
    """Engine for training & evaluating GMF model"""
    def __init__(self, config):
        self.model = MLP(config)
        if config['use_cuda'] is True:
            use_cuda(True, config['device_id'])
            self.model.cuda()
        super(MLPEngine, self).__init__(config)
        print(self.model)

        if config['pretrain']:
            self.model.load_pretrain_weights()

In [ ]:
class NeuMF(torch.nn.Module):
    def __init__(self, config):
        super(NeuMF, self).__init__()
        self.config = config
        self.num_users = config['num_users']
        self.num_items = config['num_items']
        self.latent_dim_mf = config['latent_dim_mf']
        self.latent_dim_mlp = config['latent_dim_mlp']

        self.embedding_user_mlp = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim_mlp)
        self.embedding_item_mlp = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim_mlp)
        self.embedding_user_mf = torch.nn.Embedding(num_embeddings=self.num_users, embedding_dim=self.latent_dim_mf)
        self.embedding_item_mf = torch.nn.Embedding(num_embeddings=self.num_items, embedding_dim=self.latent_dim_mf)

        self.fc_layers = torch.nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(config['layers'][:-1], config['layers'][1:])):
            self.fc_layers.append(torch.nn.Linear(in_size, out_size))

        self.affine_output = torch.nn.Linear(in_features=config['layers'][-1] + config['latent_dim_mf'], out_features=1)
        self.logistic = torch.nn.Sigmoid()

    def forward(self, user_indices, item_indices):
        user_embedding_mlp = self.embedding_user_mlp(user_indices)
        item_embedding_mlp = self.embedding_item_mlp(item_indices)
        user_embedding_mf = self.embedding_user_mf(user_indices)
        item_embedding_mf = self.embedding_item_mf(item_indices)

        mlp_vector = torch.cat([user_embedding_mlp, item_embedding_mlp], dim=-1)  # the concat latent vector
        mf_vector =torch.mul(user_embedding_mf, item_embedding_mf)

        for idx, _ in enumerate(range(len(self.fc_layers))):
            mlp_vector = self.fc_layers[idx](mlp_vector)
            mlp_vector = torch.nn.ReLU()(mlp_vector)

        vector = torch.cat([mlp_vector, mf_vector], dim=-1)
        logits = self.affine_output(vector)
        rating = self.logistic(logits)
        return rating

    def init_weight(self):
        pass

    def load_pretrain_weights(self):
        """Loading weights from trained MLP model & GMF model"""
        config = self.config
        config['latent_dim'] = config['latent_dim_mlp']
        mlp_model = MLP(config)
        if config['use_cuda'] is True:
            mlp_model.cuda()
        resume_checkpoint(mlp_model, model_dir=config['pretrain_mlp'], device_id=config['device_id'])

        self.embedding_user_mlp.weight.data = mlp_model.embedding_user.weight.data
        self.embedding_item_mlp.weight.data = mlp_model.embedding_item.weight.data
        for idx in range(len(self.fc_layers)):
            self.fc_layers[idx].weight.data = mlp_model.fc_layers[idx].weight.data

        config['latent_dim'] = config['latent_dim_mf']
        gmf_model = GMF(config)
        if config['use_cuda'] is True:
            gmf_model.cuda()
        resume_checkpoint(gmf_model, model_dir=config['pretrain_mf'], device_id=config['device_id'])
        self.embedding_user_mf.weight.data = gmf_model.embedding_user.weight.data
        self.embedding_item_mf.weight.data = gmf_model.embedding_item.weight.data

        self.affine_output.weight.data = 0.5 * torch.cat([mlp_model.affine_output.weight.data, gmf_model.affine_output.weight.data], dim=-1)
        self.affine_output.bias.data = 0.5 * (mlp_model.affine_output.bias.data + gmf_model.affine_output.bias.data)


class NeuMFEngine(Engine):
    """Engine for training & evaluating GMF model"""
    def __init__(self, config):
        self.model = NeuMF(config)
        if config['use_cuda'] is True:
            use_cuda(True, config['device_id'])
            self.model.cuda()
        super(NeuMFEngine, self).__init__(config)
        print(self.model)

        if config['pretrain']:
            self.model.load_pretrain_weights()

In [ ]:
!mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
gmf_config = {'alias': 'gmf_factor8neg4-implict',
              'num_epoch': 200,
              'batch_size': 1024,
              # 'optimizer': 'sgd',
              # 'sgd_lr': 1e-3,
              # 'sgd_momentum': 0.9,
              # 'optimizer': 'rmsprop',
              # 'rmsprop_lr': 1e-3,
              # 'rmsprop_alpha': 0.99,
              # 'rmsprop_momentum': 0,
              'optimizer': 'adam',
              'adam_lr': 1e-3,
              'num_users': 6040,
              'num_items': 3706,
              'latent_dim': 8,
              'num_negative': 4,
              'l2_regularization': 0, # 0.01
              'use_cuda': True,
              'device_id': 0,
              'model_dir':'/content/checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
              }

mlp_config = {'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
              'num_epoch': 200,
              'batch_size': 256,  # 1024,
              'optimizer': 'adam',
              'adam_lr': 1e-3,
              'num_users': 6040,
              'num_items': 3706,
              'latent_dim': 8,
              'num_negative': 4,
              'layers': [16,64,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
              'l2_regularization': 0.0000001,  # MLP model is sensitive to hyper params
              'use_cuda': True,
              'device_id': 7,
              'pretrain': False,
            #   'pretrain_mf': '/content/checkpoints/{}'.format('gmf_factor8neg4_Epoch100_HR0.6391_NDCG0.2852.model'),
              'model_dir':'/content/checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}

neumf_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 200,
                'batch_size': 1024,
                'optimizer': 'adam',
                'adam_lr': 1e-3,
                'num_users': 6040,
                'num_items': 3706,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'num_negative': 4,
                'layers': [16,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
                'l2_regularization': 0.01,
                'use_cuda': True,
                'device_id': 7,
                'pretrain': False,
                # 'pretrain_mf': 'checkpoints/{}'.format('gmf_factor8neg4_Epoch100_HR0.6391_NDCG0.2852.model'),
                # 'pretrain_mlp': 'checkpoints/{}'.format('mlp_factor8neg4_Epoch100_HR0.5606_NDCG0.2463.model'),
                'model_dir':'/content/checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'
                }

# Load Data
ml1m_dir = '/content/drive/MyDrive/Colab Notebooks/RecSys/movielens_datasets/ml-1m/ratings.dat'
ml1m_rating = pd.read_csv(ml1m_dir, sep='::', header=None, names=['uid', 'mid', 'rating', 'timestamp'],  engine='python')

# Reindex
user_id = ml1m_rating[['uid']].drop_duplicates().reindex()
user_id['userId'] = np.arange(len(user_id))
ml1m_rating = pd.merge(ml1m_rating, user_id, on=['uid'], how='left')
item_id = ml1m_rating[['mid']].drop_duplicates()
item_id['itemId'] = np.arange(len(item_id))
ml1m_rating = pd.merge(ml1m_rating, item_id, on=['mid'], how='left')
ml1m_rating = ml1m_rating[['userId', 'itemId', 'rating', 'timestamp']]
print('Range of userId is [{}, {}]'.format(ml1m_rating.userId.min(), ml1m_rating.userId.max()))
print('Range of itemId is [{}, {}]'.format(ml1m_rating.itemId.min(), ml1m_rating.itemId.max()))

# DataLoader for training
sample_generator = SampleGenerator(ratings=ml1m_rating)
evaluate_data = sample_generator.evaluate_data

# Specify the exact model
config = gmf_config
engine = GMFEngine(config)
# config = mlp_config
# engine = MLPEngine(config)
# config = neumf_config
# engine = NeuMFEngine(config)
for epoch in range(config['num_epoch']):
    print('Epoch {} starts !'.format(epoch))
    print('-' * 80)
    train_loader = sample_generator.instance_a_train_loader(config['num_negative'], config['batch_size'])
    engine.train_an_epoch(train_loader, epoch_id=epoch)
    hit_ratio, ndcg = engine.evaluate(evaluate_data, epoch_id=epoch)
    engine.save(config['alias'], epoch, hit_ratio, ndcg)

Range of userId is [0, 6039]
Range of itemId is [0, 3705]
Epoch 0 starts !
--------------------------------------------------------------------------------
[Training Epoch 0] Batch 0, Loss 0.634957492351532
[Training Epoch 0] Batch 100, Loss 0.6084944009780884
[Training Epoch 0] Batch 200, Loss 0.5835074186325073
[Training Epoch 0] Batch 300, Loss 0.5762701034545898
[Training Epoch 0] Batch 400, Loss 0.570959746837616
[Training Epoch 0] Batch 500, Loss 0.5489323139190674
[Training Epoch 0] Batch 600, Loss 0.532582700252533
[Training Epoch 0] Batch 700, Loss 0.5126482844352722
[Training Epoch 0] Batch 800, Loss 0.5320024490356445
[Training Epoch 0] Batch 900, Loss 0.538666844367981
[Training Epoch 0] Batch 1000, Loss 0.49481943249702454
[Training Epoch 0] Batch 1100, Loss 0.4894544184207916
[Training Epoch 0] Batch 1200, Loss 0.48130548000335693
[Training Epoch 0] Batch 1300, Loss 0.5132834911346436
[Training Epoch 0] Batch 1400, Loss 0.5031135082244873
[Training Epoch 0] Batch 1500, Lo

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[Evluating Epoch 0] HR = 0.0978, NDCG = 0.0428
Epoch 1 starts !
--------------------------------------------------------------------------------
[Training Epoch 1] Batch 0, Loss 0.46698641777038574
[Training Epoch 1] Batch 100, Loss 0.5061224102973938
[Training Epoch 1] Batch 200, Loss 0.5128269195556641
[Training Epoch 1] Batch 300, Loss 0.4776607155799866
[Training Epoch 1] Batch 400, Loss 0.5141273736953735
[Training Epoch 1] Batch 500, Loss 0.49529266357421875
[Training Epoch 1] Batch 600, Loss 0.5304645299911499
[Training Epoch 1] Batch 700, Loss 0.4937954545021057
[Training Epoch 1] Batch 800, Loss 0.5020802021026611
[Training Epoch 1] Batch 900, Loss 0.5221403241157532
[Training Epoch 1] Batch 1000, Loss 0.5045718550682068
[Training Epoch 1] Batch 1100, Loss 0.4829954206943512
[Training Epoch 1] Batch 1200, Loss 0.5141868591308594
[Training Epoch 1] Batch 1300, Loss 0.502382755279541
[Training Epoch 1] Batch 1400, Loss 0.502103328704834
[Training Epoch 1] Batch 1500, Loss 0.4719

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[Evluating Epoch 1] HR = 0.1018, NDCG = 0.0452
Epoch 2 starts !
--------------------------------------------------------------------------------
[Training Epoch 2] Batch 0, Loss 0.5183069109916687
[Training Epoch 2] Batch 100, Loss 0.4943470060825348
[Training Epoch 2] Batch 200, Loss 0.4967399537563324
[Training Epoch 2] Batch 300, Loss 0.47916972637176514
[Training Epoch 2] Batch 400, Loss 0.4874839186668396
[Training Epoch 2] Batch 500, Loss 0.46967369318008423
[Training Epoch 2] Batch 600, Loss 0.5061054825782776
[Training Epoch 2] Batch 700, Loss 0.49123767018318176
[Training Epoch 2] Batch 800, Loss 0.4934993088245392
[Training Epoch 2] Batch 900, Loss 0.48308172821998596
[Training Epoch 2] Batch 1000, Loss 0.49626249074935913
[Training Epoch 2] Batch 1100, Loss 0.46662434935569763
[Training Epoch 2] Batch 1200, Loss 0.5198988914489746
[Training Epoch 2] Batch 1300, Loss 0.5275518894195557
[Training Epoch 2] Batch 1400, Loss 0.4987758994102478
[Training Epoch 2] Batch 1500, Loss 

...

[Evluating Epoch 145] HR = 0.6404, NDCG = 0.3699
Epoch 146 starts !
--------------------------------------------------------------------------------
[Training Epoch 146] Batch 0, Loss 0.23054412007331848
[Training Epoch 146] Batch 100, Loss 0.2599155604839325
[Training Epoch 146] Batch 200, Loss 0.28558194637298584
[Training Epoch 146] Batch 300, Loss 0.256491482257843
[Training Epoch 146] Batch 400, Loss 0.2615330219268799
[Training Epoch 146] Batch 500, Loss 0.282105028629303
[Training Epoch 146] Batch 600, Loss 0.2488754838705063
[Training Epoch 146] Batch 700, Loss 0.2321268767118454
[Training Epoch 146] Batch 800, Loss 0.24030408263206482
[Training Epoch 146] Batch 900, Loss 0.24086397886276245
[Training Epoch 146] Batch 1000, Loss 0.2622703015804291
[Training Epoch 146] Batch 1100, Loss 0.24565011262893677
[Training Epoch 146] Batch 1200, Loss 0.2624000310897827
[Training Epoch 146] Batch 1300, Loss 0.2298111766576767
[Training Epoch 146] Batch 1400, Loss 0.21160316467285156
[Tra

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[Evluating Epoch 146] HR = 0.6397, NDCG = 0.3692
Epoch 147 starts !
--------------------------------------------------------------------------------
[Training Epoch 147] Batch 0, Loss 0.2728981375694275
[Training Epoch 147] Batch 100, Loss 0.245124951004982
[Training Epoch 147] Batch 200, Loss 0.2601642310619354
[Training Epoch 147] Batch 300, Loss 0.27080976963043213
[Training Epoch 147] Batch 400, Loss 0.2492445707321167
[Training Epoch 147] Batch 500, Loss 0.2461923509836197
[Training Epoch 147] Batch 600, Loss 0.2533149719238281
[Training Epoch 147] Batch 700, Loss 0.23905861377716064
[Training Epoch 147] Batch 800, Loss 0.256037175655365
[Training Epoch 147] Batch 900, Loss 0.2641780972480774
[Training Epoch 147] Batch 1000, Loss 0.28650274872779846
[Training Epoch 147] Batch 1100, Loss 0.27511608600616455
[Training Epoch 147] Batch 1200, Loss 0.29982149600982666
[Training Epoch 147] Batch 1300, Loss 0.2577553391456604
[Training Epoch 147] Batch 1400, Loss 0.2508794665336609
[Trai

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


[Evluating Epoch 147] HR = 0.6386, NDCG = 0.3672
Epoch 148 starts !
--------------------------------------------------------------------------------
[Training Epoch 148] Batch 0, Loss 0.28386950492858887
[Training Epoch 148] Batch 100, Loss 0.2980727553367615
[Training Epoch 148] Batch 200, Loss 0.2602621018886566
[Training Epoch 148] Batch 300, Loss 0.2421620786190033
[Training Epoch 148] Batch 400, Loss 0.2843427062034607
[Training Epoch 148] Batch 500, Loss 0.26240140199661255
[Training Epoch 148] Batch 600, Loss 0.2718813121318817
[Training Epoch 148] Batch 700, Loss 0.2593681514263153
[Training Epoch 148] Batch 800, Loss 0.25852370262145996
[Training Epoch 148] Batch 900, Loss 0.25290027260780334
[Training Epoch 148] Batch 1000, Loss 0.2834629714488983
[Training Epoch 148] Batch 1100, Loss 0.28946444392204285
[Training Epoch 148] Batch 1200, Loss 0.25377339124679565
[Training Epoch 148] Batch 1300, Loss 0.2657666802406311
[Training Epoch 148] Batch 1400, Loss 0.26753664016723633
[